In [1]:
import time
import pprint
import numpy as np
import DRL.takeoff_dqn as tfdqn
import LLC 
import scaffold.fgdata as dfer
import scaffold.pidpilot as PID
import datetime
import fgmodule.fgenv as fgenv
import pandas as pd
import os
from scaffold.timer import gettime
import gym

'''
##########controlframe############


##########state_dict##############
{'aileron': 0.0, 'elevator': 0.0, 'rudder': -0.026, 'flaps': 0.0, 'throttle0': 0.6, 'throttle1': 0.6, 'vsi-fpm': 0.0, 'alt-ft': -372.808502, 'ai-pitch': 0.401045, 'ai-roll': 0.050598, 'ai-offset': 0.0, 'hi-heading': 80.568947, 'roll-deg': 0.050616, 'pitch-deg': 0.401055, 'heading-deg': 90.013458, 'airspeed-kt': 71.631187, 'speed-north-fps': -0.021637, 'speed-east-fps': 119.609383, 'speed-down-fps': -0.071344, 'uBody-fps': 119.606964, 'vBody-fps': -0.005778, 'wBody-fps': 0.765776, 'north-accel-fps_sec': 0.118887, 'east-accel-fps_sec': 5.870498, 'down-accel-fps_sec': -0.003219, 'x-accel-fps_sec': 6.095403, 'y-accel-fps_sec': -0.148636, 'z-accel-fps_sec': -32.113453, 'latitude': 21.325245, 'longitude': -157.93947, 'altitude': 22.297876, 'crashed': 0.0}
'''

actions_list = [
    'a_aileron',  # 副翼 控制飞机翻滚 [-1,1]
    'a_elevator',  # 升降舵 控制飞机爬升 [-1,1]
    'a_rudder',  # 方向舵 控制飞机转弯（地面飞机方向控制） [-1,1]
    'a_throttle0',  # 油门0 [0,1]
    'a_throttle1'  # 油门1 [0,1]
    # 'flaps',  # 襟翼 在飞机起降过程中增加升力，阻力 [0,1],实测影响不大，而且有速度限制
    #TODO: 方向舵调整片
]
LLC_FEATURES = [
    'pitch-deg',  # 飞机俯仰角
    'roll-deg',  # 飞机滚转角
    'heading-deg',  # 飞机朝向
    'vsi-fpm',  # 爬升速度
    'uBody-fps',  # 飞机沿机身X轴的速度
    'vBody-fps',  # 飞机沿机身Y轴的速度
    'wBody-fps',  # 飞机沿机身Z轴的速度
    'x-accel-fps_sec',  # 飞机沿机身X轴的加速度
    'y-accel-fps_sec',  # 飞机沿机身Y轴的加速度
    'z-accel-fps_sec',  # 飞机沿机身z轴的加速度
]


LLC_GOALS = {
    'pitch-deg': 0.0,  # 飞机俯仰角
    'roll-deg': 0.0,  # 飞机滚转角
    'heading-deg': 90.0,  # 飞机朝向
    'vsi-fpm': 0.0,  # 爬升速度
    'uBody-fps': 120.0,  # 飞机沿机身X轴的速度
    'vBody-fps': 0.0,  # 飞机沿机身Y轴的速度
    'wBody-fps': 0.0,  # 飞机沿机身Z轴的速度
    'x-accel-fps_sec': 5.0,  # 飞机沿机身X轴的加速度
    'y-accel-fps_sec': 0.0,  # 飞机沿机身Y轴的加速度
    'z-accel-fps_sec': 0.0,  # 飞机沿机身z轴的加速度
}

LLC_FEATURE_BOUNDS = {
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
    'vsi-fpm': [0., 10.0],  # 爬升速度
    'uBody-fps': [0., 600.],  # 飞机沿机身X轴的速度
    'vBody-fps': [-200., 200.],  # 飞机沿机身Y轴的速度
    'wBody-fps': [-200., 200.],  # 飞机沿机身Z轴的速度
    'x-accel-fps_sec': [0., 50.],  # 飞机沿机身X轴的加速度
    'y-accel-fps_sec': [-30., 30.],  # 飞机沿机身Y轴的加速度
    'z-accel-fps_sec': [-300., 300.],  # 飞机沿机身z轴的加速度
}

LLC_ACTIONS = [
    'aileron',  # 副翼 控制飞机翻滚 [-1,1]
    'elevator',  # 升降舵 控制飞机爬升 [-1,1]
    'rudder',  # 方向舵 控制飞机转弯（地面飞机方向控制） [-1,1]
    'throttle0',  # 油门0 [0,1]
    'throttle1'  # 油门1 [0,1]
    # 'flaps',  # 襟翼 在飞机起降过程中增加升力，阻力 [0,1],实测影响不大，而且有速度限制
    #TODO: 方向舵调整片
]
DATA_ACTIONS = [
    'a_aileron',  # 副翼 控制飞机翻滚 [-1,1]
    'a_elevator',  # 升降舵 控制飞机爬升 [-1,1]
    'a_rudder',  # 方向舵 控制飞机转弯（地面飞机方向控制） [-1,1]
    'a_throttle0',  # 油门0 [0,1]
    'a_throttle1'  # 油门1 [0,1]
]
LLC_ACTION_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'elevator': [-1, 1],  # 升降舵 控制飞机爬升 [-1,1] up/down
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'throttle0': [0, 1],  # 油门0
    'throttle1': [0, 1],  # 油门1
    # 'flaps': [0, 0]  # 襟翼 在飞机起降过程中增加升力，阻力  Key[ / ]	Extend / retract flaps
    #TODO: 方向舵调整片
}

DATA_BOUNDS = {
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
    'vsi-fpm': [0., 10.0],  # 爬升速度
    'uBody-fps': [0., 600.],  # 飞机沿机身X轴的速度
    'vBody-fps': [-200., 200.],  # 飞机沿机身Y轴的速度
    'wBody-fps': [-200., 200.],  # 飞机沿机身Z轴的速度
    'x-accel-fps_sec': [0., 50.],  # 飞机沿机身X轴的加速度
    'y-accel-fps_sec': [-30., 30.],  # 飞机沿机身Y轴的加速度
    'z-accel-fps_sec': [-300., 300.],  # 飞机沿机身z轴的加速度
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'elevator': [-1, 1],  # 升降舵 控制飞机爬升 [-1,1] up/down
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'throttle0': [0, 1],  # 油门0
    'throttle1': [0, 1],  # 油门1
    'flaps': [0, 1],  # 襟翼 在飞机起降过程中增加升力，阻力  Key[ / ]	Extend / retract flaps
    #TODO: 方向舵调整片
    'a_aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'a_elevator': [-1, 1],  # 升降舵 控制飞机爬升 [-1,1] up/down
    'a_rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'a_throttle0': [0, 1],  # 油门0
    'a_throttle1': [0, 1],  # 油门1
}


def fgstart(fg2client_addr=("127.0.0.1", 5700), client2fg_addr=("127.0.0.1", 5701), telnet_addr=("127.0.0.1", 5555)):
    '''
    简化FG通信启动过程
    ---
    Inputs:
        - fg2client_addr
        - client2fg_addr
        - telnet_addr
    Outputs:
        - myfgenv:初始话完成的fgenv类

    '''
    myfgenv = fgenv.fgenv(telnet_addr, fg2client_addr, client2fg_addr)
    # initial_state = myfgenv.initial()
    myfgenv.initial()

    return myfgenv


MAX_EPISODE = 1000
MAX_EP_STEPS = 200

GAMMA = 0.9
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic

##
epoch = 100
step = 300

In [2]:
target_states = dfer.load_target_state("data/traindata")

In [3]:
myfgenv = fgstart()

# bounds = {
#     'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
# }

flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 23 : 9 : 1
fgudp stop begin! waiting for 2 thread quit
send data to 127.0.0.1:5701 stop!UDP rece on 127.0.0.1:5700 stop!

reset完成 23 : 9 : 2
flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 23 : 9 : 57


In [20]:
# myllc.sess.close()

In [4]:
myllc = LLC.LLC(LLC_FEATURE_BOUNDS,LLC_FEATURE_BOUNDS,LLC_ACTION_BOUNDS)

In [5]:
state = myfgenv.replay()

replay to ground 完成!time:23 : 10 : 14
auto start完成 23 : 10 : 17


In [6]:
state

{'aileron': 0.0,
 'elevator': 0.0,
 'rudder': 0.0,
 'flaps': 0.0,
 'throttle0': 0.0,
 'throttle1': 0.0,
 'vsi-fpm': 0.0,
 'alt-ft': -372.246063,
 'ai-pitch': 0.503405,
 'ai-roll': 0.179295,
 'ai-offset': 0.0,
 'hi-heading': 82.918945,
 'roll-deg': 0.056098,
 'pitch-deg': 0.466483,
 'heading-deg': 90.002332,
 'airspeed-kt': 1.532065,
 'speed-north-fps': -0.000307,
 'speed-east-fps': 0.048647,
 'speed-down-fps': 0.00012,
 'uBody-fps': 0.048645,
 'vBody-fps': 0.000305,
 'wBody-fps': 0.000516,
 'north-accel-fps_sec': 7.4e-05,
 'east-accel-fps_sec': -0.05512,
 'down-accel-fps_sec': -0.000414,
 'x-accel-fps_sec': 0.206657,
 'y-accel-fps_sec': -0.031552,
 'z-accel-fps_sec': -32.152016,
 'latitude': 21.325247,
 'longitude': -157.943136,
 'altitude': 21.357961,
 'crashed': 0.0}

In [7]:
goal = dfer.get_target_state(state, target_states)
goal

{'pitch-deg': 0.443287,
 'roll-deg': 0.056379999999999986,
 'heading-deg': 90.000221,
 'vsi-fpm': 0.0,
 'uBody-fps': 4.114077,
 'vBody-fps': 0.000356,
 'wBody-fps': 0.028794,
 'x-accel-fps_sec': 2.742848,
 'y-accel-fps_sec': -0.031711,
 'z-accel-fps_sec': -32.134853}

In [15]:
myllc.goals

{'pitch-deg': [-90.0, 90.0],
 'roll-deg': [-180.0, 180.0],
 'heading-deg': [0.0, 360.0],
 'vsi-fpm': [0.0, 10.0],
 'uBody-fps': [0.0, 600.0],
 'vBody-fps': [-200.0, 200.0],
 'wBody-fps': [-200.0, 200.0],
 'x-accel-fps_sec': [0.0, 50.0],
 'y-accel-fps_sec': [-30.0, 30.0],
 'z-accel-fps_sec': [-300.0, 300.0]}

In [8]:
myllc.choose_action(state,goal)

(array([-1.        , -0.09898096, -1.        ,  0.5       , -0.5       ]),
 array([-1.        , -0.09898096, -1.        ,  1.        ,  0.        ]))

In [10]:
import importlib
importlib.reload(LLC)

<module 'LLC' from 'E:\\project\\flightgear\\FGAutopilot\\LLC.py'>

In [13]:
 for e in range(epoch):
        state = myfgenv.replay()
        time.sleep(2)

        goal_count = 0
        goal = dfer.get_target_state(state, target_states)
        next_goal = goal

        ep_reward = 0

        for s in range(step):

            goal_count +=1

            action,action_true = myllc.choose_action(state,goal)

            action_frame = dfer.action2frame(action_true)
            next_state, reward , done , info = myfgenv.step(action_frame) 
            
            r_ = LLC.llc_reward(state , goal, reward)

            if goal_count%5 ==0:
                next_goal = dfer.get_target_state(next_state, target_states)

            myllc.learn(state, goal, r_, action,next_state , next_goal)
            
            state = next_state
            goal = next_goal

            ep_reward += r_
            if done:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                break
            
            if s == step-1:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                # if ep_reward > -300:RENDER = True
                break
            time.sleep(0.1)

replay to ground 完成!time:23 : 15 : 30
auto start完成 23 : 15 : 33
Episode: 0  Reward: -1407 Explore: 3.00
replay to ground 完成!time:23 : 16 : 10
auto start完成 23 : 16 : 13
Episode: 1  Reward: -1283 Explore: 3.00
replay to ground 完成!time:23 : 16 : 50
auto start完成 23 : 16 : 53
Episode: 2  Reward: -1210 Explore: 3.00
replay to ground 完成!time:23 : 17 : 30
auto start完成 23 : 17 : 33
Episode: 3  Reward: -1115 Explore: 3.00
replay to ground 完成!time:23 : 18 : 10
auto start完成 23 : 18 : 13
Episode: 4  Reward: -1332 Explore: 3.00
replay to ground 完成!time:23 : 18 : 50
auto start完成 23 : 18 : 53
Episode: 5  Reward: -991 Explore: 3.00
replay to ground 完成!time:23 : 19 : 30
auto start完成 23 : 19 : 33
Episode: 6  Reward: -1108 Explore: 3.00
replay to ground 完成!time:23 : 20 : 10
auto start完成 23 : 20 : 13
Episode: 7  Reward: -1249 Explore: 3.00
replay to ground 完成!time:23 : 20 : 50
auto start完成 23 : 20 : 53
Episode: 8  Reward: -1288 Explore: 3.00
replay to ground 完成!time:23 : 21 : 28
auto start完成 23 : 21 : 31
E

auto start完成 0 : 5 : 41
Episode: 79  Reward: -1960 Explore: 0.00
replay to ground 完成!time:0 : 6 : 17
auto start完成 0 : 6 : 20


KeyboardInterrupt: 

In [38]:
myllc.ddpg.pointer

25081

In [37]:
t1= time.clock()
myllc.learn(state, goal, r_, action,next_state , next_goal)
print(time.clock()-t1)

0.0027973283603017762
